<a href="https://colab.research.google.com/github/DARKINOO/Pytorch_practice/blob/main/rnn_pytorch_qa_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
#tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()


In [3]:
tokenize('What is capital city of France?')

['what', 'is', 'capital', 'city', 'of', 'france']

In [4]:
#vocab
vocab = {'<UNK>':0}

In [9]:
def build_vocab(row):
  tokenized_ques = tokenize(row['question'])
  tokenized_ans = tokenize(row['answer'])

  merged_tokens = tokenized_ques + tokenized_ans

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)



In [10]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [11]:
len(vocab)

324

In [12]:
#convert words to numerical indexs
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
       indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [14]:
text_to_indices("What is Darkino", vocab)

[1, 2, 0]

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

In [16]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)


In [17]:
dataset = QADataset(df, vocab)

In [19]:
dataset[10]

(tensor([ 1,  2,  3,  4,  5, 53]), tensor([54]))

In [30]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [31]:
#RNN architecture
import torch.nn as nn

In [32]:
class SimpleRNN(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_ques = self.embedding(question)
    hidden, final = self.rnn(embedded_ques)
    output = self.fc(final.squeeze(0))
    return output


In [33]:
learning_rate = 0.001
epochs = 20


In [34]:
model = SimpleRNN(len(vocab))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
#training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:
    optimizer.zero_grad()

    #forwad pass
    output = model(question)
    #loss
    loss = criterion(output, answer[0])
    #gradients
    loss.backward()

    #update
    optimizer.step()

    total_loss+=loss.item()

  print(f"Epoch {epoch+1},Loss: {total_loss:4f}")

Epoch 1,Loss: 525.759835
Epoch 2,Loss: 461.452696
Epoch 3,Loss: 385.686063
Epoch 4,Loss: 320.264866
Epoch 5,Loss: 267.403822
Epoch 6,Loss: 217.677343
Epoch 7,Loss: 173.192404
Epoch 8,Loss: 134.460376
Epoch 9,Loss: 103.029531
Epoch 10,Loss: 79.490172
Epoch 11,Loss: 61.179179
Epoch 12,Loss: 48.332658
Epoch 13,Loss: 38.489433
Epoch 14,Loss: 31.390854
Epoch 15,Loss: 25.914396
Epoch 16,Loss: 21.479206
Epoch 17,Loss: 18.130277
Epoch 18,Loss: 15.574894
Epoch 19,Loss: 13.376417
Epoch 20,Loss: 11.695233


In [37]:
def predict(model, question, threshold=0.5):
  #convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  #tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  #send to model
  output = model(question_tensor)

  #convert logics to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  #find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [38]:
predict(model,"What is capital of France?")

paris


In [40]:
predict(model,"What is Golang?")

I don't know
12


In [44]:
predict(model,"What is largest planet in our Solar system?")

jupiter
